# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.115824e+02     1.492370e+00
 * time: 0.43640995025634766
     1     1.064050e+01     9.363666e-01
 * time: 1.1786160469055176
     2    -1.108785e+01     1.059030e+00
 * time: 1.2555978298187256
     3    -3.353590e+01     7.167854e-01
 * time: 1.3629920482635498
     4    -4.679525e+01     5.545151e-01
 * time: 1.488800048828125
     5    -5.672393e+01     2.303792e-01
 * time: 1.5722389221191406
     6    -5.963109e+01     1.722818e-01
 * time: 1.6371710300445557
     7    -6.085781e+01     7.658288e-02
 * time: 1.7123630046844482
     8    -6.139268e+01     3.505146e-02
 * time: 1.7874929904937744
     9    -6.166074e+01     3.183408e-02
 * time: 1.973127841949463
    10    -6.181361e+01     2.836616e-02
 * time: 2.0494790077209473
    11    -6.194784e+01     1.777265e-02
 * time: 2.124145030975342
    12    -6.203082e+01     1.435931e-02
 * time: 2.199025869369507
    13    -6.207963e+01     1.293358e-02
 * time: 2.2773969173431

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671065
    AtomicLocal         -18.8557659
    AtomicNonlocal      14.8522632
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485368 
    Xc                  -19.3336819

    total               -62.261666461944